In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os

# Reading data

In [ ]:
class VRPData:
    def __init__(self, name, type, dimension, edge_weight_type, capacity, node_coord_section):
        self.name = name
        self.type = type
        self.dimension = dimension
        self.edge_weight_type = edge_weight_type
        self.capacity = capacity
        self.node_coord_section = node_coord_section

